In [17]:
import numpy as np

def recommend_movies_manual(user_index, U, V, movie_titles, categories, top_n=3):
    """
    Recommend movies for a given user based on manually defined latent factors.
    This function scales the predicted ratings into the [1, 5] range for interpretability.
    
    Parameters:
      user_index (int): Index of the user.
      U (np.ndarray): User latent factor matrix.
      V (np.ndarray): Movie latent factor matrix.
      movie_titles (list of str): Titles for each movie.
      categories (list of str): Names of the latent factor categories.
      top_n (int): Number of recommendations.
      
    Returns:
      recommendations (list of dict): Each dict contains:
         - movie: Title of the recommended movie.
         - scaled_rating: The predicted (scaled) rating.
         - explanation: A brief explanation for the recommendation.
    """
    # Compute raw predicted ratings (dot product)
    raw_ratings = U[user_index, :] @ V.T
    
    # Scale raw ratings to the range [1, 5]
    min_rating = raw_ratings.min()
    max_rating = raw_ratings.max()
    if max_rating - min_rating == 0:
        # Avoid division by zero if all ratings are equal.
        scaled_ratings = np.full_like(raw_ratings, 3.0)
    else:
        scaled_ratings = 1 + 4 * (raw_ratings - min_rating) / (max_rating - min_rating)
    
    # For recommendation, assume the user hasn't rated any movies.
    # If they have, you might need to exclude those (here we assume none are rated).
    
    top_indices = np.argsort(scaled_ratings)[-top_n:][::-1]
    
    recommendations = []
    for idx in top_indices:
        rating = scaled_ratings[idx]
        # Determine the dominant latent factor (category) for the movie.
        dominant_cat_index = np.argmax(V[idx])
        explanation = (
            f"{movie_titles[idx]} is recommended because it has a predicted rating of {rating:.2f} "
            f"and is primarily associated with {categories[dominant_cat_index]}."
        )
        recommendations.append({
            "movie": movie_titles[idx],
            "scaled_rating": rating,
            "explanation": explanation
        })
    return recommendations

# Define categories (predefined latent factor names)
categories = ["Action", "Comedy", "Drama", "SciFi", "Romance"]

# Manually defined latent factors for demonstration:
U = np.array([
    [0.9, 0.05, 0.03, 0.01, 0.01],  # User 0: Primarily Action
    [0.1, 0.7, 0.1, 0.05, 0.05],
    [0.05, 0.1, 0.75, 0.05, 0.05],
    [0.1, 0.05, 0.05, 0.75, 0.05]
])
V = np.array([
    [0.85, 0.05, 0.05, 0.03, 0.02],  # Movie 0: Action
    [0.05, 0.85, 0.05, 0.03, 0.02],  # Movie 1: Comedy
    [0.05, 0.05, 0.85, 0.03, 0.02],  # Movie 2: Drama
    [0.03, 0.02, 0.05, 0.85, 0.05],  # Movie 3: SciFi
    [0.02, 0.03, 0.05, 0.05, 0.85]   # Movie 4: Romance
])
movie_titles = [
    "Blockbuster Action", 
    "Hilarious Comedy", 
    "Deep Drama", 
    "SciFi Adventure", 
    "Romantic Tale"
]



In [19]:
def recommend_movies_for_users(user_indices, U, V, movie_titles, categories, top_n=3):
    """
    Generate recommendations for multiple users.
    
    Parameters:
      user_indices (list of int): List of user indices to generate recommendations for.
      U (np.ndarray): User latent factor matrix.
      V (np.ndarray): Movie latent factor matrix.
      movie_titles (list of str): Titles for each movie.
      categories (list of str): Names of the latent factor categories.
      top_n (int): Number of recommendations per user.
      
    Returns:
      all_recommendations (dict): A dictionary mapping each user index to a list of recommendation dicts.
    """
    all_recommendations = {}
    for user_index in user_indices:
        recs = recommend_movies_manual(user_index, U, V, movie_titles, categories, top_n=top_n)
        all_recommendations[user_index] = recs
    return all_recommendations


# Define a list of user indices for which you want recommendations
user_indices = [0, 1, 2, 3]

# Get recommendations for these users
all_recs = recommend_movies_for_users(user_indices, U, V, movie_titles, categories, top_n=3)

# Print recommendations for each user
for user_idx, recs in all_recs.items():
    print(f"Recommendations for User {user_idx}:")
    for rec in recs:
        print(f"- {rec['movie']} (Scaled Predicted Rating: {rec['scaled_rating']:.2f})")
        print(f"  Explanation: {rec['explanation']}")
    print() 


Recommendations for User 0:
- Blockbuster Action (Scaled Predicted Rating: 5.00)
  Explanation: Blockbuster Action is recommended because it has a predicted rating of 5.00 and is primarily associated with Action.
- Hilarious Comedy (Scaled Predicted Rating: 1.32)
  Explanation: Hilarious Comedy is recommended because it has a predicted rating of 1.32 and is primarily associated with Comedy.
- Deep Drama (Scaled Predicted Rating: 1.24)
  Explanation: Deep Drama is recommended because it has a predicted rating of 1.24 and is primarily associated with Drama.

Recommendations for User 1:
- Hilarious Comedy (Scaled Predicted Rating: 5.00)
  Explanation: Hilarious Comedy is recommended because it has a predicted rating of 5.00 and is primarily associated with Comedy.
- Deep Drama (Scaled Predicted Rating: 1.45)
  Explanation: Deep Drama is recommended because it has a predicted rating of 1.45 and is primarily associated with Drama.
- Blockbuster Action (Scaled Predicted Rating: 1.45)
  Expla